# Advisories

> **Unstructured Data**: 8 security advisory markdown documents
   - Located in the `advisories/` directory
   - Each contains detailed vulnerability descriptions, code examples, attack vectors, remediation steps, and CVSS breakdown tables
   - Covers vulnerabilities like XSS, SQL Injection, RCE, CSRF, SSRF, Path Traversal, etc.


In [1]:
from pathlib import Path

advisories_dir = Path("../data/advisories").resolve()
advisory_files = sorted(advisories_dir.glob("*.md"))

print(f"Found {len(advisory_files)} advisory files:")
for f in advisory_files:
    print(f"  - {f.name}")

Found 8 advisory files:
  - advisory-001.md
  - advisory-002.md
  - advisory-003.md
  - advisory-004.md
  - advisory-005.md
  - advisory-006.md
  - advisory-007.md
  - advisory-008.md


In [2]:
from snyk_ai.utils.markdown import parse_markdown_document, BlockType

parsed = {}

for advisory_path in advisory_files:
    blocks = parse_markdown_document(advisory_path)
    
    print(f"{advisory_path.name}: {len(blocks)} blocks\n")

    assert blocks[0].type is BlockType.HEADER
    assert blocks[0].content.startswith("Security Advisory: ")
    print(f"  {blocks[0].content.removeprefix('Security Advisory: ')}\n")

    assert blocks[2].type is BlockType.HEADER
    assert blocks[2].content == "Executive Summary"
    assert blocks[3].type is BlockType.PARAGRAPH
    print(f"  {blocks[3].content}\n")

    parsed[advisory_path.name] = blocks
    



advisory-001.md: 34 blocks

  Cross-Site Scripting (XSS) in express-validator

  A critical Cross-Site Scripting (XSS) vulnerability has been discovered in the `express-validator` package affecting versions prior to 4.5.0. This vulnerability allows attackers to inject malicious JavaScript code through validation error messages, potentially compromising user sessions and sensitive data.

advisory-002.md: 37 blocks

  SQL Injection in webapp-auth

  A critical SQL injection vulnerability has been identified in the `webapp-auth` package affecting versions 2.0.0 through 2.3.0. This vulnerability allows attackers to execute arbitrary SQL commands through the authentication query builder, potentially leading to complete database compromise, data exfiltration, and authentication bypass.

advisory-003.md: 37 blocks

  Dependency Confusion in secure-config

  A dependency confusion vulnerability has been discovered in the `secure-config` package affecting versions 3.0.0 through 3.1.9. This vuln